##Install package

Package pyspark tidak termasuk package default dalam google colab. Setiap kali kita memulai session, kita perlu menginstal package pyspark.

Kita akan lakukan instalasi dengan menggunakan pip (python install package)

In [7]:
%pip install pyspark

##Spark Session

Aplikasi PySpark diawali dengan inisialisasi SparkSession. SparkSession diperkenalkan sejak versi Spark 2.0. SparkSession merupakan pintu masuk ke eksekusi Spark, untuk membuat objek-objek Spark RDD, DataFrame, dan DataSet.


In [8]:
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder.appName('Belajar Spark').getOrCreate()

DataFrame dapat dibuat dengan banyak cara, di antaranya :
- Dari python object, misalnya array/list, tuple, dictionary, pandas dataframe, dll. Digunakan dengan perintah createDataFrame().
- Dari file : csv, json, dll. Digunakan dengan perintah read.
- Dari file di HDFS, dengan perintah read ke lokasi HDFS.
- Dari RDD, dengan perintah toDF().
- dll.

###Membuat DataFrame dari python tuple

In [26]:
mydata = (('apel',100),('jeruk',200),('pisang',150),('pir',50))

df_from_tuple = spark.createDataFrame(mydata, ["nama", "jumlah"])

df_from_tuple.show()

+------+------+
|  nama|jumlah|
+------+------+
|  apel|   100|
| jeruk|   200|
|pisang|   150|
|   pir|    50|
+------+------+



###Membuat DataFrame dari python list

In [28]:
mydata = [['apel',100],['jeruk',200],['pisang',150],['pir',50]]

df_from_list = spark.createDataFrame(mydata,["nama", "jumlah"])

df_from_list.show()

+------+------+
|  nama|jumlah|
+------+------+
|  apel|   100|
| jeruk|   200|
|pisang|   150|
|   pir|    50|
+------+------+



###Membuat DataFrame dari pandas dataframe

In [12]:
import pandas as pd

df_pd = pd.DataFrame(list(zip(['apel','jeruk','pisang','pir'],
                              [100,200,150,100])),
                  columns =['nama','jumlah'])

df = spark.createDataFrame(df_pd)
df.show()

+------+------+
|  nama|jumlah|
+------+------+
|  apel|   100|
| jeruk|   200|
|pisang|   150|
|   pir|   100|
+------+------+



####Membuat DataFrame dari file csv

In [29]:
!wget https://raw.githubusercontent.com/urfie/Seri-Belajar-PySpark/main/buah.csv

namafile = 'buah.csv'

--2023-07-05 12:15:51--  https://raw.githubusercontent.com/urfie/Seri-Belajar-PySpark/main/buah.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115 [text/plain]
Saving to: ‘buah.csv’

buah.csv            100%[===================>]     115  --.-KB/s    in 0s      

2023-07-05 12:15:51 (4.69 MB/s) - ‘buah.csv’ saved [115/115]



In [30]:
df = spark.read.csv('buah.csv', header=True)
df.show()

+------+------+------+
|  nama| warna|jumlah|
+------+------+------+
|  apel| merah|   100|
|  apel| hijau|   150|
|pisang|kuning|   200|
|pisang| hijau|   250|
| jeruk| orens|   100|
| jeruk| hijau|   300|
+------+------+------+



##Eksplorasi DataFrame

Sebelum melakukan pemrosesan, biasanya kita melakukan eksplorasi untuk mengetahui karakteristik data yang akan diproses. Beberapa perintah yang biasa kita gunakan yaitu :


####Menampilkan satu baris pertama

In [13]:
df.first()

Row(nama='apel', jumlah=100)

####Menampilkan beberapa baris pertama

In [14]:
df.show(2)

+-----+------+
| nama|jumlah|
+-----+------+
| apel|   100|
|jeruk|   200|
+-----+------+
only showing top 2 rows



####Menampilkan jumlah baris

In [15]:
df.count()

4

####Menampilkan kolom

In [25]:
df.columns

['nama', 'jumlah']

####Menampilkan jumlah kolom

In [24]:
len(df.columns)

2

####Menampilkan skema

In [16]:
df.printSchema()

root
 |-- nama: string (nullable = true)
 |-- jumlah: long (nullable = true)



####Menampilkan rangkuman statistik

In [17]:
df.describe().show()

+-------+------+------------------+
|summary|  nama|            jumlah|
+-------+------+------------------+
|  count|     4|                 4|
|   mean|  null|             137.5|
| stddev|  null|47.871355387816905|
|    min|  apel|               100|
|    max|pisang|               200|
+-------+------+------------------+



###Memilih kolom DataFrame (Subset DataFrame)

Untuk memilih kolom tertentu saja, gunakan perintah` select(namakolom)`

In [18]:
df.select("nama").show()

+------+
|  nama|
+------+
|  apel|
| jeruk|
|pisang|
|   pir|
+------+



###Memilih baris (filtering)

Perintah `filter(kondisi)` digunakan untuk memilih atau memfilter baris sebuah DataFrame

In [19]:
df.filter(df.jumlah > 100).show()

+------+------+
|  nama|jumlah|
+------+------+
| jeruk|   200|
|pisang|   150|
+------+------+



Untuk menggunakan kondisi berupa operasi string, dapat digunakan fungsi-fungsi dari pyspark.sql.Column yang terkait string, misalnya `contains(), startswith(), endswith()`


In [20]:
df.filter(df.nama.contains('is')).show(5)
df.filter(df.nama.startswith('je')).show(5)

+------+------+
|  nama|jumlah|
+------+------+
|pisang|   150|
+------+------+

+-----+------+
| nama|jumlah|
+-----+------+
|jeruk|   200|
+-----+------+



Kita juga dapat menggunakan fungsi `like()` yang serupa dengan SQL statement like

In [21]:
df.filter(df.nama.like('pi%')).show(5)

+------+------+
|  nama|jumlah|
+------+------+
|pisang|   150|
|   pir|   100|
+------+------+

